In [138]:

import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
SEED = 1013
np.random.seed(SEED)
#nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples 
from utils import *
#from parameters import *
from nltk.stem import PorterStemmer
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout,Concatenate,Dense, Embedding, LSTM, SpatialDropout1D, Flatten, GRU, Bidirectional, Conv1D, Input,MaxPooling1D
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
stemmer = PorterStemmer()
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
stopwords_english = stopwords.words('english')

In [139]:
a_file = open("vocabulary.json", "r")
vocabulary = json.load(a_file)
len(vocabulary['mohammed'])

5280

In [204]:
train_data_file = '/home/parush/stance/Experiments/stance_mohammed/train.txt'
test_data_file = '/home/parush/stance/Experiments/stance_mohammed/test.txt'
TARGETS = [ 'Atheism','Climate Change is a Real Concern', 'Feminist Movement','Hillary Clinton', 'Legalization of Abortion' ]


In [205]:
# train_data_file = '/home/parush/stance/Experiments/SomasundaranWiebe-politicalDebates/train.txt'
# test_data_file = '/home/parush/stance/Experiments/SomasundaranWiebe-politicalDebates/test.txt'
# TARGETS = ['god','healthcare','guns','gayRights','abortion', 'creation']

In [206]:
# train_data_file = '/home/parush/stance/Experiments/Data_MPCHI/train.txt'
# test_data_file = '/home/parush/stance/Experiments/Data_MPCHI/test.txt'
# TARGETS = ['Are E-Cigarettes safe?','Does MMR Vaccine lead to autism in children?',
#      'Does Sunlight exposure lead to skin cancer?','Does Vitamin C prevent common cold?',
#      'Should women take HRT post-menopause?']

In [207]:
def process_tweet(tweet):
    '''
    Input: 
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet
    
    '''
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    ### START CODE HERE ###
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and # remove stopwords
            word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
    ### END CODE HERE ###
    return tweets_clean

def train_and_test():
    
    sentence_maxlen = 909 #set 909 only when loading biGRUCNN or biLSTMCNN trained on debates set.
    x_train = []
    y_train = []
    all_favor_tweets = []
    all_against_tweets = []
    
    with open(train_data_file, 'r') as trainfile:
        for line in trainfile:
            
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
            
            if line[0].strip() != 'ID' and line[3].strip() == 'FAVOR':
                tweet = line[2]
                tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                all_favor_tweets.append(tweet)
            elif line[0].strip() != 'ID' and line[3].strip() == 'AGAINST':
                tweet = line[2]
                tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                all_against_tweets.append(tweet)
            
    x_train = all_favor_tweets + all_against_tweets
    y_train = np.append(np.ones(len(all_favor_tweets)), np.zeros(len(all_against_tweets))) 
    
    
    x_test = []
    y_test = []
    all_favor_tweets_test = []
    all_against_tweets_test = []
    with open(test_data_file, 'r') as testfile:
        for line in testfile:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
        

            if line[0] != 'ID' and line[3] == 'FAVOR':
                tweet = line[2]
                tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                all_favor_tweets_test.append(tweet)
            elif line[0].strip() != 'ID' and line[3].strip() == 'AGAINST':
                tweet = line[2]
                tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                all_against_tweets_test.append(tweet)

    x_test = all_favor_tweets_test + all_against_tweets_test
    y_test = np.append(np.ones(len(all_favor_tweets_test)), np.zeros(len(all_against_tweets_test)))
    
    return x_train, y_train, x_test, y_test, sentence_maxlen
                
            
def tweet_to_tensor(processed_tweet, vocab_dict, unk_token="__UNK__"):
    tensor = []
    unk_ID = vocab_dict[unk_token]
    for word in processed_tweet:
        word_ID = vocab_dict[word] if word in vocab_dict else unk_ID
        tensor.append(word_ID)
    return tensor                
            
def load_embeddings(embedding,dim):
    if embedding == 'twitter':
        path = '/home/parush/stance/Experiments/embeddings/twitter/glove.twitter.27B.'+str(dim)+'d.txt'
    else:
        path ='/home/parush/stance/Experiments/embeddings/wikipedia/glove.6B.'+str(dim)+'d.txt'
    word_embeddings = {}
    with open(path, 'r') as f:
        for each_emb in f:
            emb = each_emb.split(' ')
            word_embeddings[emb[0]] = np.asarray(emb[1:], dtype='float32')
    return word_embeddings
def get_embeddings(embedding,dim):
    if embedding == 'twitter':
        embedding_matrix_twitter = np.zeros((vocab_size, dim))
        word_embeddings_twitter = load_embeddings(embedding, dim)
        print(embedding_matrix_twitter[0])
        for each_word,index in Vocab.items():
            if each_word in word_embeddings_twitter:
                embedding_matrix_twitter[index] = word_embeddings_twitter[each_word]
        return embedding_matrix_twitter
    else:
        embedding_matrix_wikipedia = np.zeros((vocab_size, dim))
        word_embeddings_wikipedia = load_embeddings(embedding, dim)
        for each_word,index in Vocab.items():
            if each_word in word_embeddings_wikipedia:
                embedding_matrix_wikipedia[index] = word_embeddings_wikipedia[each_word]
        return embedding_matrix_wikipedia
        
                        

In [208]:
x_train, y_train, x_test, y_test, sentence_maxlen = train_and_test()

In [209]:
for i in range(len(x_train)):
    tweet_tensor = tweet_to_tensor(x_train[i], vocabulary['debates']) # Set dic of dataset of trained model that you will load
    if len(tweet_tensor) < sentence_maxlen:
        diff = sentence_maxlen - len(tweet_tensor)
        n_pad = [0]*diff
        tweet_tensor = tweet_tensor + n_pad
    x_train[i] = tweet_tensor
for i in range(len(x_test)):
    tweet_tensor = tweet_to_tensor(x_test[i], vocabulary['debates']) # Set dic of dataset of trained model that you will load
    if len(tweet_tensor) < sentence_maxlen:
        diff = sentence_maxlen - len(tweet_tensor)
        n_pad = [0]*diff
        tweet_tensor = tweet_tensor + n_pad
    x_test[i] = tweet_tensor

In [210]:
x_train = np.array(x_train)

x_test = np.array(x_test)
 
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [211]:
print("Sentence maxlength {},".format(sentence_maxlen))

Sentence maxlength 909,


In [212]:
x_test.shape

(1019, 909)

In [213]:
s_model = tf.keras.models.load_model('saved_model/debates/biLSTMCNN')

In [214]:
y_pred = np.round(s_model.predict(x_test))
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

         0.0     0.7083    0.3329    0.4529       715
         1.0     0.3016    0.6776    0.4174       304

    accuracy                         0.4357      1019
   macro avg     0.5050    0.5052    0.4352      1019
weighted avg     0.5870    0.4357    0.4423      1019



In [470]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []

for train, val in kfold.split(x_train, y_train):
    
    
    model.fit(x_train[train], y_train[train], epochs = 1, batch_size = 16, verbose=2)
    scores = model.evaluate(x_train[val], y_train[val], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

108/108 - 3s - loss: 0.6329 - accuracy: 0.6624
accuracy: 72.76%
108/108 - 3s - loss: 0.5180 - accuracy: 0.7595
accuracy: 77.56%
108/108 - 3s - loss: 0.3952 - accuracy: 0.8272
accuracy: 87.44%
108/108 - 3s - loss: 0.2934 - accuracy: 0.8889
accuracy: 92.46%
108/108 - 3s - loss: 0.2088 - accuracy: 0.9241
accuracy: 95.44%
85.13% (+/- 8.67%)


              precision    recall  f1-score   support

         0.0     0.4430    0.7688    0.5621       930
         1.0     0.5897    0.2558    0.3568      1208

    accuracy                         0.4790      2138
   macro avg     0.5163    0.5123    0.4595      2138
weighted avg     0.5259    0.4790    0.4461      2138

